In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
import time
class Timer:
    def __init__(self):
        self.CURTIME = 0
    def dtime(self, msg):
        o, self.CURTIME = time.time_ns()/1000-self.CURTIME, time.time_ns()/1000
        tf.print(f"mark: {msg}, time since called: {o}")

In [33]:
class Node(dict):
    def __init__(self, *args, **kwds):
        super(Node, self).__init__(*args, **kwds)
        self.__dict__ = self
    
    def __call__(self, x):
        i = [0,0]
        if not isinstance(x, list):
            x = list(x)
        if isinstance(self.left, int):
            i[0] = x[self.left]
        else:
            i[0] = self.left(x)
        if isinstance(self.right, int):
            i[1] = x[self.right]
        else:
            i[1] = self.right(x)
        return self.op(i, self.weights)
    
    def __getitem__(self, key):
        if key == 0:
            return self.left
        elif key == 1:
            return self.right
        else:
            raise IndexError(f"Tree has only 2 branches, can't get branch {key}")

    def __setitem__(self, key, value):
        if key == 0:
            self.left = value
        elif key == 1:
            self.right = value
        else:
            raise IndexError(f"Tree has only 2 branches,  can't set branch {key}")
    
    def __hash__(self):
        return hash(
            str(hash(self.op))
            +str(hash(self.weights))
            +str(hash(self.left))
            +str(hash(self.right))
        )
    
    def __repr__(self):
        w = self.weights
        eq = f"{w[0]}x^2 + {w[1]}x + {w[2]}xy + {w[3]}y + {w[4]}y^2 + {w[5]}"
        return ' '.join(('( eq:', eq, 'l:', str(self.left), 'r:', str(self.right), ')'))
        


In [67]:
def lse(weights, test_inputs, test_results):
    r = []
    for p, y in zip(test_inputs, test_results):
        r.append((bino(p, weights)-y)**2)
    return sum(r)/len(r)

In [77]:
def node_generator(node, test_inputs, test_results):
    i = []
    for point in test_inputs:
        i.append([])
        if isinstance(node.left, int):
            i[-1].append(point[node.left])
        else:
            i[-1].append(node.left(test_inputs))
        if isinstance(node.right, int):
            i[-1].append(point[node.right])
        else:
            i[-1].append(node.right(test_inputs))
    i = tf.constant(i, dtype=tf.float32)
    test_results = tf.constant(test_results, dtype=tf.float32)
    def inner(weights):
        r = tf.TensorArray(tf.float32, size=i.shape[0])
        for p in tf.range(i.shape[0]):
            r = r.write(p, 
            (
                weights[0]*i[p][0]**2 +
                weights[1]*i[p][0] + 
                weights[2]*i[p][0]*i[p][1] +
                weights[3]*i[p][1] +
                weights[4]*i[p][1]**2 +
                weights[5]
            ))
        r = r.stack()
        r = r - test_results
        r = r**2
        return tf.reduce_sum(r)
    return inner

In [88]:
t_f = lambda x: 1/8*x[0]**2 + 3/8*x[0] + 2/8*x[0]*x[1]
t_p = [(random.random()*2-10, random.random()*2-10) for _ in range(50)]
t_y = list(map(t_f,t_p))

In [105]:
import random

MAX_ITERATION = 1000
MAX_GRAD = 1
MAX_ERR = 1
GSF = 10000
timer = Timer()
def gradient_descent_node(node, test_inputs, test_results, descent_rate):
    timer.dtime("init")
    #dw = tf.constant([descent_rate]*6, dtype=tf.float32)
    #ELEMENTARY = tf.eye(6, dtype=tf.float32)*dw+tf.ones([6,6])
    grad = 999
    err = 999
    step = 0
    n = node_generator(node, test_inputs, test_results)
    weights = tf.Variable([random.random() for _ in range(6)], dtype=tf.float32)
    while tf.abs(grad) > MAX_GRAD and err > MAX_ERR:
        timer.dtime(f"step {step}")
        with tf.GradientTape() as t:
            err = n(weights)
        J = t.jacobian(err, weights)
        timer.dtime(f"jacob computed")
        grad = tf.reduce_sum(J)
        err = tf.reduce_sum(err)
        J = J/tf.sqrt(tf.reduce_sum(J**2)) #normalize Jacobi matrix
        scale = 1-GSF/(GSF+tf.abs(grad))
#       err0 = lse(weights, test_inputs, test_results)
#       for i in tf.range(6):
#           tf.print(weights*ELEMENTARY[i])
#           err = err.write(i, (lse(weights*ELEMENTARY[i], test_inputs, test_results)-err0)/dw[i])
#       err = err.stack()
        weights.assign_sub(scale*descent_rate*J)
        step += 1
        timer.dtime(f"step: {step}, J: {J}, err: {err}, grad: {grad}, descent speed: {scale*descent_rate}")
        if step > MAX_ITERATION:
            break
    return {"error": err, "grad": grad, "weights": weights}

In [106]:
max(t_y)-min(t_y)

10.721016179748418

In [107]:
bino = lambda x, w: x[0]**2*w[0] + x[0]*w[1] + x[0]*x[1]*w[2] + x[1]**2*w[4] + x[1]*w[3] + w[5]
n = Node(left=0, right=1, op=bino, weights=[0]*6)

In [ ]:
g = gradient_descent_node(n, t_p, t_y, 1e-1)
n.weights = g["weights"]
n.err = g["error"]

mark: init, time since called: 1666776813830700.5
mark: step 0, time since called: 14862.5


In [ ]:
n

In [ ]:
from matplotlib import pyplot as plt

r_x = np.linspace(-10, 10)
r_y1 = [t_f([p, 0]) for p in r_x]
r_y2 = [t_f([p, -5]) for p in r_x]
r_y3 = [t_f([p, 5]) for p in r_x]
n_y1 = [n([p, 0]) for p in r_x]
n_y2 = [n([p, -5]) for p in r_x]
n_y3 = [n([p, 5]) for p in r_x]

plt.plot(
    r_x, r_y1, "r--",
    r_x, r_y2, "g--",
    r_x, r_y3, "b--",
    r_x, n_y1,  "ro",
    r_x, n_y2,  "go",
    r_x, n_y2,  "bo"
)

In [ ]:
проверить, что вызывает ретрейс на каждой итерации. Скорее всего, это node_generator. Проверить, перетрассируется ли функция. Попробовать ускорить jacobian